In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import glob
from pathlib import Path

## QC pills box detection 

    This notebook is a work for pill box dent detection using edge detection and hough transform. Combinded with finding curves on the edges lines in picture to check if the box is in good shape.

In [3]:
def load_images_from_directory(directory):
    
    path = Path(directory)
    path = path.glob("*.png")

    images = []

    # Iterate over all files with .png extension in the directory
    for imagepath in path:
        img = cv2.imread(str(imagepath))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
        images.append(img)

    return images

directory1 = "archive/damaged/side" # Path to the directory containing the images
images1 = load_images_from_directory(directory1)
print(len(images1), "images in the first test set")


100 images in the first test set


In [4]:
def edge_detection(image, sigma=0.33):
    # Convert the image to grayscale
    v = np.median(image)
    lower = int(max(100, (1.0 - sigma) * v))
    upper = int(min(130, (1.0 + sigma) * v))

    # Apply edge detection
    edges = cv2.Canny(image, lower, upper)
    return edges





In [5]:
#find if there is a curve edges on the image
def find_curved_edges(edges, min_angle=55, max_angle=160):
    # Define the Hough transform parameters
    rho = 0.77 # distance resolution in pixels of the Hough grid
    theta = np.pi / 180 # angular resolution in radians of the Hough grid
    threshold = 70 # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 70 # minimum number of pixels making up a line
    max_line_gap = 5 # maximum gap in pixels between connectable line segments

    # Run Hough on the edge-detected image
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    # Iterate over the output "lines" and draw them on the image
    for line in lines:
        for x1, y1, x2, y2 in line:
            angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
            if min_angle <= abs(angle) <= max_angle:
                return True
    return False




In [6]:
#loop from a list images and find the curved edges by feeding in one at a time
count = 0
for image in images1:
    edges = edge_detection(image)
    if find_curved_edges(edges):
        if True:
            count += 1
            
print(count, "images with curved edges in the test set")



73 images with curved edges in the test set


## OCR with easyocr

    The OCR is done using tessaract to read the text on the pills box. The text is then used to check if the box is in tru lable according to file name.

In [59]:
import easyocr

# Create an OCR reader
reader = easyocr.Reader(['en'])


# Get a set of three from directory and feed them to the OCR model
directory2 = "archive/intact/side" # Path to the directory containing the images
images2 = load_images_from_directory(directory2)
print(len(images2), "images in the second test set")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


100 images in the second test set


In [60]:
def get_text_from_ocr(image):
    # Assuming 'reader' is properly defined elsewhere
    result = reader.readtext(image)
    num_result = [detection[1] for detection in result if detection[1].isdigit() and len(detection[1]) == 13]
    return num_result


In [63]:
#function compare file name with the result from OCR
def compare_filename_with_ocr(images, ocr_result):
    count = 0
    for i, image in enumerate(images):
        filename = Path(directory2).glob("*.png")
        filename = [f for f in filename]
        filename = filename[i].stem
        if filename in ocr_result:
            count += 1
    return count

In [64]:
count = 0
for image in images2:
    result = get_text_from_ocr(image)
    #compare result with the file name
    count += compare_filename_with_ocr(images2, result)
    

In [65]:
print(count, "images with matching filenames in the test set")

0 images with matching filenames in the test set


# Conclusion
    - On the first part of the notebook we have a good detection of the damaged box .
    - On the second part we did not succeed in reading out the correct SN number due to too much sheer in picture.